In [1]:
import pandas as pd

In [2]:
# import all_cities_analysis.csv
df = pd.read_csv('../data/all_cities_analysis.csv')

In [3]:
df.head()

,bank,categoryName,city,totalScore,rank,cid,publishedAtDate,reviewsCount,reviewsDistribution,textTranslated,reviewId,reviewerId,reviewerNumberOfReviews,stars,lat,lng,cleaned_text,sentiment
0,Attijariwafa Bank,Banque,Agadir,1.8,119,1.589799e+19,2023-02-24,5,"{'oneStar': 4, 'twoStar': 0, 'threeStar': 0, '...","Outstanding service, especially if you ask for...",ChdDSUhNMG9nS0VJQ0FnSUNodDhhd3pRRRAB,1.052275e+20,1.0,1.0,30.41016,-9.559908,outstanding service especially ask assistance ...,4.0
1,Attijariwafa Bank,Banque,Agadir,1.8,119,1.589799e+19,2023-02-21,5,"{'oneStar': 4, 'twoStar': 0, 'threeStar': 0, '...","Deplorable service, every time I come to this ...",ChZDSUhNMG9nS0VJQ0FnSUNoazVqWmVREAE,1.023462e+20,1.0,1.0,30.41016,-9.559908,deplorable service every time come agency prep...,1.0
2,Attijariwafa Bank,Banque,Agadir,1.8,119,1.589799e+19,2023-01-06,5,"{'oneStar': 4, 'twoStar': 0, 'threeStar': 0, '...",Very good service by the director who helped m...,ChZDSUhNMG9nS0VJQ0FnSUNCcDdPeUdnEAE,1.110533e+20,2.0,5.0,30.41016,-9.559908,good service director helped hour fairly diffi...,4.0
3,Attijariwafa Bank,Banque,Agadir,1.8,119,1.589799e+19,2020-11-13,5,"{'oneStar': 4, 'twoStar': 0, 'threeStar': 0, '...",Zero is bad telephone service,ChZDSUhNMG9nS0VJQ0FnSUNpdzdhWk1REAE,1.031901e+20,1.0,1.0,30.41016,-9.559908,zero bad telephone service,1.0
4,Attijariwafa Bank,Banque,Agadir,1.8,119,1.589799e+19,2020-10-05,5,"{'oneStar': 4, 'twoStar': 0, 'threeStar': 0, '...",Zero service,ChdDSUhNMG9nS0VJQ0FnSURDbGMtcDZnRRAB,1.016100e+20,45.0,1.0,30.41016,-9.559908,zero service,1.0


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17206 entries, 0 to 17205
Data columns (total 18 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   bank                     17206 non-null  object 
 1   categoryName             17205 non-null  object 
 2   city                     17206 non-null  object 
 3   totalScore               17206 non-null  float64
 4   rank                     17206 non-null  int64  
 5   cid                      17206 non-null  float64
 6   publishedAtDate          17206 non-null  object 
 7   reviewsCount             17206 non-null  int64  
 8   reviewsDistribution      17206 non-null  object 
 9   textTranslated           9257 non-null   object 
 10  reviewId                 17206 non-null  object 
 11  reviewerId               17206 non-null  float64
 12  reviewerNumberOfReviews  17206 non-null  float64
 13  stars                    17206 non-null  float64
 14  lat                   

In [5]:
# see textTranslated, stars and sentiment columns
df[['textTranslated', 'stars', 'sentiment']].head()

,textTranslated,stars,sentiment
0,"Outstanding service, especially if you ask for...",1.0,4.0
1,"Deplorable service, every time I come to this ...",1.0,1.0
2,Very good service by the director who helped m...,5.0,4.0
3,Zero is bad telephone service,1.0,1.0
4,Zero service,1.0,1.0


In [25]:
import psycopg2

# Establish a connection to your PostgreSQL database
conn = psycopg2.connect(database="bank_reviews_dw", user="postgres", password="yasir", host="localhost", port="5432")
cur = conn.cursor()


In [26]:
# create Review table (fact table) having as columns: reviewId, published_at_date,cid,id_loc,sentiment
cur.execute('''CREATE TABLE IF NOT EXISTS review(
    reviewId SERIAL PRIMARY KEY,
    published_at_date DATE,
    cid VARCHAR(255),
    id_loc VARCHAR(255),
    sentiment VARCHAR(255)
)''')


In [27]:
# visualize the table
cur.execute('''SELECT * FROM review LIMIT 5''')
cur.fetchall()

[]

In [31]:
# Iterate over each row in the DataFrame and insert data into the Reviews table
for _, row in df.iterrows():
    cur.execute("INSERT INTO review (published_at_date, cid, sentiment) VALUES (%s, %s, %s)",
                (row['publishedAtDate'], row['cid'], row['sentiment']))

# Fetch and print the first 5 rows from the Reviews table
cur.execute("SELECT * FROM review LIMIT 5")
rows = cur.fetchall()
for row in rows:
    print(row)

(1, datetime.date(2023, 2, 24), '15897994659992177000', None, '4.0')
(2, datetime.date(2023, 2, 21), '15897994659992177000', None, '1.0')
(3, datetime.date(2023, 1, 6), '15897994659992177000', None, '4.0')
(4, datetime.date(2020, 11, 13), '15897994659992177000', None, '1.0')
(5, datetime.date(2020, 10, 5), '15897994659992177000', None, '1.0')


In [32]:
# Commit the changes
conn.commit()

# Close the cursor and the connection
cur.close()
conn.close()